# Parte 5:

____________

##  Data Visualization tramite grafici Choropleth Map:

Per concludere la nostra analisi utilizzeremo un file *geojson* (vedi [qui](https://it.wikipedia.org/wiki/GeoJSON)) per elaborare delle mappe in cui, tramite una scala di colori, visualizzeremo la distribuzione per provincia dei *top* studenti.\
Useremo quindi come metrica la percentuale di studenti che hanno totalizzato un punteggio maggiore del 110% della media nazionalele.\
Filtreremo i dataframe delle materie solo per l'anno 2021-22 e solo per le prove svoltesi in V liceo.\
Identificheremo quindi la provincia "migliore" e la provincia "peggiore", tra i *top studenti* per la materia in esame.\
Aggiungeremo anche una visualizzazione della percentuale di partecipazione di tutti gli studenti per ogni provincia

##### Import librerie e caricamento dati:

In [1]:
from pathlib import Path
import pandas as pd
import plotly.express as px
import pickle

Per motivi di gestione della grandezza del file notebook non *printiamo* i grafici ma li salviamo esportandoli come *.html*\
Definimiamo la funzione che crea la cartella per contenerli e li salva:

In [2]:
def export_html(fig, full_path: Path) -> None:
    outdir = full_path.parent
    if not Path.is_dir(outdir):
        Path.mkdir(outdir)
    fig.write_html(full_path)
    #fig.write_image(full_path)

In [3]:
DATA_PATH = "../data/in/"
DATA_PATH_PLOTS = "../data/plots/"

Tramite *pickle* apriamo il file che abbiamo salvato alla fine della **Parte 3&4** che contiene i dataframe già filtrati per materia-anno-grado da utilizzare per la *data-visualization*:

In [4]:
with open(Path(DATA_PATH, "dizionario_df_materie.pickle"), "rb") as fp:
    dizionario_df_materie = pickle.load(fp)

In [5]:
df_geo_mat_21 = dizionario_df_materie["df_geo_mat_21"]
df_geo_ita_21 = dizionario_df_materie["df_geo_ita_21"]
df_geo_ing_21 = dizionario_df_materie["df_geo_ing_21"]

Creiamo una lista dei dataframe e una delle materie per utilizzarle come iterabili al fine della visualizzazione:

In [6]:
lista_df = [df_geo_mat_21, df_geo_ita_21, df_geo_ing_21]
materie = ["Matematica", "Italiano", "Inglese"]

In [7]:
import json
GEOJSON_PATH = Path(DATA_PATH, "province.geojson")

In [8]:
with open(GEOJSON_PATH, 'r') as fp:
    province_geojson = json.load(fp)

In [9]:
#PER CREARE IL CAMPO ID CHE CONTIENE LA SIGLA DELLA PROVINCIA DA USARE COME CHIAVE NEL GEOJSON:
for feature_ in province_geojson['features']:
    feature_['id'] = feature_['properties']['SIGLA']

In [10]:
for i in range(len(lista_df)):
    if i != 2:
        minore=lista_df[i][lista_df[i]["maggiore_110"].eq(lista_df[i]["maggiore_110"].min())]["nome_provincia"]
        maggiore=lista_df[i][lista_df[i]["maggiore_110"].eq(lista_df[i]["maggiore_110"].max())]["nome_provincia"]
        fig = px.choropleth_mapbox(
            data_frame=lista_df[i],
            geojson=province_geojson,
            locations='sigla_provincia',
            color="maggiore_110",
            color_continuous_scale="rdbu",
            range_color=(lista_df[i]["maggiore_110"].min(), lista_df[i]["maggiore_110"].max()),
            #hover_data=['tot_sezioni', 'sezioni_corso', 'moda_volume', 'moda_editore'],
            mapbox_style="carto-positron",
            zoom=4, 
            center={"lat": 41.9, "lon": 12.0},
            opacity=0.5,
            title="Percentuali studenti punteggio superiore al 110% della media prove Invalsi {} 5°Liceo 2021:<br><sup>La regione più performante del 2021 è stata {}</sup><br><sup>La regione meno performante del 2021 è stata {}</sup>".format(materie[i],maggiore.values[0],minore.values[0]),
            labels={'maggiore_110':'% Studenti > 110%'}
            )
        
        #fig.show()
        #fig.write_image("images/fig1.webp")
        export_html(fig, Path(DATA_PATH_PLOTS, "{}.html".format(materie[i])))
    else:
        minore=lista_df[i][lista_df[i]["livello_3"].eq(lista_df[i]["livello_3"].min())]["nome_provincia"]
        maggiore=lista_df[i][lista_df[i]["livello_3"].eq(lista_df[i]["livello_3"].max())]["nome_provincia"]
        fig = px.choropleth_mapbox(
            data_frame=lista_df[i],
            geojson=province_geojson,
            locations='sigla_provincia',
            color="livello_3",
            color_continuous_scale="rdbu",
            range_color=(lista_df[i]["livello_3"].min(), lista_df[i]["livello_3"].max()),
            #hover_data=['tot_sezioni', 'sezioni_corso', 'moda_volume', 'moda_editore'],
            mapbox_style="carto-positron",
            zoom=4, 
            center={"lat": 41.9, "lon": 12.0},
            opacity=0.5,
            title="Percentuali studenti livello B2  prove Invalsi Inglese 5°Liceo 2021:<br><sup>La regione più performante del 2021 è stata {}</sup><br><sup>La regione meno performante del 2021 è stata {}</sup>".format(maggiore.values[0],minore.values[0]),
            labels={'livello_3':'% Studenti B2'} 
        )
        #fig.show()
        export_html(fig, Path(DATA_PATH_PLOTS, "inglese.html"))

In [11]:
for i in range(len(lista_df)):
    minore=lista_df[i][lista_df[i]["perc_copertura_stu"].eq(lista_df[i]["perc_copertura_stu"].min())]["nome_provincia"]
    fig = px.choropleth_mapbox(
        data_frame=lista_df[i],
        geojson=province_geojson,
        locations='sigla_provincia',
        color="perc_copertura_stu",
        color_continuous_scale="rdbu",
        range_color=(lista_df[i]["perc_copertura_stu"].min(), lista_df[i]["perc_copertura_stu"].max()),
        #hover_data=['tot_sezioni', 'sezioni_corso', 'moda_volume', 'moda_editore'],
        mapbox_style="carto-positron",
        zoom=4, 
        center={"lat": 41.9, "lon": 12.0},
        opacity=0.5,
        title="Percentuali partecipazione studenti prove {} 5°Liceo 2021:<br><sup>Provincia con la minore partecipazione {}</sup>".format(materie[i],minore.values[0]),
        #title="Percentuali partecipazione studenti prove {} 5°Liceo 2021".format(materie[i]),
        labels={'perc_copertura_stu':'% copertura'}
    )
    #fig.show()
    export_html(fig, Path(DATA_PATH_PLOTS, "partecipazione_{}.html".format(materie[i])))

------------

Da quanto appare nei grafici così ottenuti, la distribuzione degli studenti *top* si concentra nel Nord Italia.
Anche se abbiamo visualizzato solo quella porzione di studenti che riescono ad ottenere un punteggio superiore al 110% della media nazionale, studenti comunque preparatissimi nell'affrontare le prove *INVALSI*, sembra palese manifestarsi quel classico **divario Nord-Sud** che caratterizza il *Belpaese*!